In [15]:
# Import geometry features, NGSolve and xfem
from netgen.geom2d import SplineGeometry
from netgen.occ import *   # Opencascade for geometry modeling
from ngsolve import *
from xfem import *
from math import pi
from xfem.mlset import *
# Visualisation
from ngsolve.webgui import *
DrawDC = MakeDiscontinuousDraw(Draw)

interactive = 1

In [21]:
# Quadrilateral (or simplicial mesh)
quad_mesh = False
# Mesh diameter
h0 = 0.01

In [26]:
outer = Rectangle(1, 1).Face()
outer.edges.name="outer"
outer.edges.Max(X).name = "r"
outer.edges.Min(X).name = "l"
outer.edges.Min(Y).name = "b"
outer.edges.Max(Y).name = "t"

inner = MoveTo(0, 0.4).Rectangle(0.8, 0.2).Face()
inner.edges.name="interface"
outer = outer - inner

inner.faces.name="inner"
inner.faces.col = (1, 0, 0)
outer.faces.name="outer"

# Draw(outer)

mesh = Mesh(OCCGeometry(outer, dim=2).GenerateMesh(maxh=h0))
# Draw(mesh);
print(mesh.GetBoundaries())

('l', 'interface', 'interface', 'interface', 'l', 't', 'r', 'b')


In [27]:
line1 = y - 2*x
line2 = y + 2*x -1.6
line3 = y + 2*x -1
line4 = y - 2*x + 0.6
level_sets = (line1, line2, line3, line4)
nr_ls = len(level_sets)
lsets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

for i, lset_p1 in enumerate(lsets_p1):
    InterpolateToP1(level_sets[i],lset_p1)
    # DrawDC(lset_p1,-1,1,mesh)

In [29]:
# omega = ~DomainTypeArray((NEG,NEG,ANY,ANY))
omega = ~DomainTypeArray((NEG,NEG,POS,POS))
DrawDC(omega.Indicator(lsets_p1),-1,1,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [25]:
# 使用区域类型组合初始化 DomainTypeArray
# 比如选取第一个levelset的任意区域和第二个levelset的负区域
domain = DomainTypeArray((ANY,NEG)) 

# 但是注意这里 ANY 不是一个具体的区域，是一个 COMBINED_DOMAIN_TYPES， 所以不能直接用于积分
# 而是要展开成每个单独的区域再去积分
# 因此可以查看这里domain的所有区域list
# 结果表示第一个levelset的任意区域其实就是既包括正的部分，也包括负的部分
print(domain.as_list)

# 也可以查看区域的余维数
print(domain.codim)

[(<DOMAIN_TYPE.POS: 1>, <DOMAIN_TYPE.NEG: 0>), (<DOMAIN_TYPE.NEG: 0>, <DOMAIN_TYPE.NEG: 0>)]
0


## DomainTypeArray 可以当成“几何集合”来做布尔运算
- DomainTypeArray = 多个 level set 的符号组合（NEG/POS/IF）构成的几何区域
- 可以对它做 NOT、UNION、INTERSECTION：
    - ~ （NOT）：取补集
    - \| （UNION）：取并集
    - &（INTERSECTION）：取交集 （注意取交集可能会增加codimension）

In [27]:
circles = DomainTypeArray((NEG,NEG))
computation_domain = ~circles
computation_domain.as_list

[(<DOMAIN_TYPE.POS: 1>, <DOMAIN_TYPE.NEG: 0>),
 (<DOMAIN_TYPE.POS: 1>, <DOMAIN_TYPE.POS: 1>),
 (<DOMAIN_TYPE.NEG: 0>, <DOMAIN_TYPE.POS: 1>)]

## 生成边界

In [30]:
boundary = circles.Boundary()
boundary.as_list

[(<DOMAIN_TYPE.NEG: 0>, <DOMAIN_TYPE.IF: 2>),
 (<DOMAIN_TYPE.IF: 2>, <DOMAIN_TYPE.NEG: 0>)]

## 可视化
DomainTypeArray 描述的是“区域”，可以使用 .Indicator(lsets)处理codim=0的区域 和 .IndicatorSmoothed(lsets,eps)处理codim>0的区域。

这些函数返回一个CoefficientFunction满足：
- 区域内部 = 1
- 区域外部 = 0
- 对于 codim > 0（面、线、点），.IndicatorSmoothed() 会显示一个值为1的“带状区域”。
这两个函数主要用于 调试（debugging） 和 可视化（visualization）。

In [32]:
DrawDC(circles.Indicator(lsets_p1),-1,1,mesh,'circles_indicator')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [33]:
DrawDC(boundary.IndicatorSmoothed(lsets_p1, 0.01), -3.5, 2.5, mesh,
       "boundary_indicator")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

## 去除不必要的区域

In [ ]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
neg = ci.GetElementsOfType(NEG)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(neg),mesh)
# Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


In [ ]:
print(type(ba_facets))

In [ ]:
vtk = VTKOutput(mesh,[levelset,BitArrayCF(hasneg),BitArrayCF(hasif)],['levelset','active_mesh','boundary'],"/mnt/d/ngs_output/Biot_Brinkman/heart_domain",subdivision=0)
# vtk = VTKOutput(mesh,[levelset_domain],['levelset_domain'],"/mnt/d/ngs_output/Biot_Brinkman/levelset_domain",subdivision=0)
# vtk = VTKOutput(mesh,[BitArrayCF(neg)],['uncutElements'],"/mnt/d/ngs_output/Biot_Brinkman/uncutElements",subdivision=0)
# vtk = VTKOutput(mesh,[BitArrayCF(ba_facets)],['ghost_faces'],"/mnt/d/ngs_output/Biot_Brinkman/ghost_faces",subdivision=0)
vtk.Do()   #输出网格

In [ ]:
print(help(Draw))